In [1]:
from sklearn.metrics import confusion_matrix
from tqdm.notebook import tqdm as tqdm
from keras.models import load_model
from keras import backend as K
from datetime import datetime

from Dental_Tool.Inception_v3 import Custom_Inception_V3, Simple_Inception
from Dental_Tool.Inception import create_inception_v4
from Dental_Tool.Data_processing import *
from Dental_Tool.Dental_Model import *
from Dental_Tool.Process_results import *
from Dental_Tool.Dataloader import *

import keras.backend as K 
import matplotlib.pyplot as plt
import numpy as np
import keras
import time

Using TensorFlow backend.


In [2]:
# Sheng_ID = ["002555 042513 x", "000408 102419 x", "005627 120209 x", "004151 091409 x",
#             "003615 010816 x", "007274 021016 x", "007274 021016 x", "004359 030716 x",
#             "001742 082712 x", "002456 060517 x", "1899 120718 x", "004499 110515 x",
#             "13529 092513 x",  "000411 112119 x", "008908 090309 x", "003262 103015 x",
#             "10689 102418 x", "003670 020718 x", "010953 031618 x", ]

In [ ]:
# all_models = { "CNN_Net": CNN_Net, "Inception": Inception, "ResNet": ResNet }
root_dir  =  "Inception_Three_new_Balance_all_Chang_Fold_5" + datetime.now().strftime("%Y%m%d-%H%M%S")

init_directory(f"Results/{root_dir}/table")

model_info = ("Inception", Inception)

performance_list = []

epochs     = 15
Fold_num = 5
classes = 3
num_params = 1
fold_data_num = 1

path = f"balance_dataset/Class_{classes}_All_Chang"


for train_dataset, valid_dataset, test_dataset, train_generator, valid_generator, test_generator in\
        prepared_data(path, classes, Fold_num, batch_size=64):
        
#         train_dataset = train_dataset[~train_dataset.ID.isin(Sheng_ID)]
#         valid_dataset = valid_dataset[~valid_dataset.ID.isin(Sheng_ID)]
#         test_dataset  = test_dataset [~test_dataset.ID.isin(Sheng_ID)]
        
#         train_generator = make_generator(train_dataset, batch_size=64)
#         valid_generator = make_generator(valid_dataset, batch_size=64)
#         test_generator  = make_generator(test_dataset, batch_size=64)
        
        
        
        K.clear_session()
        print(f"Training dataset: {len(train_dataset)},Validation dataset: {len(valid_dataset)}, Testing dataset: {len(test_dataset)}" )
        
        init_directory(f"Results/{root_dir}/Fold_{fold_data_num}/table")
           
        train_dataset.to_csv(f"Results/{root_dir}/Fold_{fold_data_num}/table/train_dataset.csv", index=False)
        valid_dataset.to_csv(f"Results/{root_dir}/Fold_{fold_data_num}/table/valid_dataset.csv", index=False)
        test_dataset.to_csv(f"Results/{root_dir}/Fold_{fold_data_num}/table/test_dataset.csv"  , index=False)
        
        K.clear_session()
        best_models = None
        
        model_name, model_fnc = model_info
        K.clear_session()

        param_dir  = f"{model_name}_Fold_{fold_data_num}"
        print(f"Fold {fold_data_num}: loading {model_name} ......")

        start = time.time()
        model = model_fnc(input_shape=(200, 180, 1), classes=len(np.unique(train_dataset["Class"])))
        end = time.time()
        elapse = end - start
        print(f"Fold {fold_data_num}: loading Done, cost {elapse} seconds")


        init_directory(f"Results/{root_dir}/Fold_{fold_data_num}/{param_dir}/parameter")

        filepath="Results/%s/Fold_%d/%s/parameter/weights-improvement-{epoch:02d}-{val_accuracy:.4f}.hdf5" \
                    %(root_dir, fold_data_num, param_dir)
        checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

        # change 
        history = model.fit_generator(
                                generator=train_generator,
                                validation_data=valid_generator,
                                epochs=epochs,
                                workers=2,
                                shuffle=True,
                                verbose=1,
                                callbacks=[checkpoint]
                               )
        plot_result(history, f"Results/{root_dir}/Fold_{fold_data_num}/{param_dir}")

        K.clear_session()
        top_k_param = get_k_top_value(f"Results/{root_dir}/Fold_{fold_data_num}/{param_dir}/parameter",  k_th = num_params )
        best_param = top_k_param[0]
        
        K.clear_session()
        print(f"loading {best_param} ......")
        start = time.time()
        best_model = load_model(best_param)
        end = time.time()
        elapse = end - start
        print(f"loading Done, cost {elapse} seconds")

        test_predict = best_model.predict_generator(test_generator, verbose=1)
        test_predict = test_predict[:len(test_dataset)]
        test_result  = np.argmax(test_predict, axis=1)
        
        test_dataset["Predict"] = test_result
        
        K.clear_session()
        
        confusion = confusion_matrix(test_dataset.Class.astype('int'), test_result.astype('int'))
        curr_acc = (test_dataset.Class == test_dataset.Predict).mean() * 100
        title = "Accuracy  = {:5.2f} %".format(curr_acc)
        print(title)
        init_directory(f"Results/{root_dir}/Fold_{fold_data_num}/table")
        plot_confusion_matrix(confusion, len(np.unique(test_dataset.Class)), curr_acc, f"Results/{root_dir}/Fold_{fold_data_num}", 0)
        
        test_predict = np.round(test_predict, 2)
        
#         test_dataset[f"{model_name}_Prob"] = pd.Series(test_predict.tolist())
        
        if classes == 3 : test_dataset[['Class_0_prob', 'Class_1_prob', 'Class_2_prob']] = pd.DataFrame(test_predict)
        elif classes == 2 : test_dataset[['Class_0_prob', 'Class_1_prob']] = pd.DataFrame(test_predict)
        
        test_dataset["Predict"] = test_result
        
        test_dataset.to_csv(f"Results/{root_dir}/Fold_{fold_data_num}/table/test_predict.csv", index=False)
        detail = statistic(test_dataset)
        detail.to_csv(f"Results/{root_dir}/Fold_{fold_data_num}/table/cmx_stage.csv")
        print(detail)
        
        
        K.clear_session()
        
        performance_list.append(curr_acc)
        fold_data_num += 1

Training dataset: 77280,Validation dataset: 26160, Testing dataset: 25680
Fold 1: loading Inception ......
Fold 1: loading Done, cost 8.252841472625732 seconds
Epoch 1/15
 227/1208 [====>.........................] - ETA: 28:18 - loss: 0.9312 - accuracy: 0.5777

In [ ]:
total_score = np.mean(performance_list)
title = "Accuracy  = {:5.2f} %".format(total_score)
title